# Produisez une étude de marché

# Scénario
Votre entreprise **d'agroalimentaire** souhaite se développer à l'international. Elle est spécialisée dans...
 le poulet !

L'international, oui, mais pour l'instant, le champ des possibles est bien large : aucun pays particulier ni aucun continent n'est pour le moment choisi. Tous les pays sont envisageables !

Votre objectif sera d'aider **à cibler plus particulièrement certains pays**, dans le but d'approfondir ensuite l'étude de marché. Plus particulièrement, l'idéal serait de produire des "groupes" de pays, plus ou moins gros, dont on connaît les caractéristiques.

Dans un premier temps, la stratégie est plutôt d'exporter les produits plutôt que de produire sur place, c'est-à-dire dans le(s) nouveau(x) pays ciblé(s).


Pour identifier les pays propices à une insertion dans le marché du poulet,
 Il vous a été demandé de cibler les pays. 
 Etudier les régimes alimentaires de chaque pays, notamment en termes de protéines d'origine animale et en termes de calories.


Construisez votre échantillon contenant l'ensemble des pays disponibles, chacun caractérisé par ces variables :

différence de population entre une année antérieure (au choix) et l'année courante, exprimée en pourcentage ;
proportion de protéines d'origine animale par rapport à la quantité totale de protéines dans la disponibilité alimentaire du pays ;
disponibilité alimentaire en protéines par habitant ;
disponibilité alimentaire en calories par habitant.

Construisez un dendrogramme contenant l'ensemble des pays étudiés, puis coupez-le afin d'obtenir 5 groupes.

Caractérisez chacun de ces groupes selon les variables cités précédemment, et facultativement selon d'autres variables que vous jugerez pertinentes (ex : le PIB par habitant). Vous pouvez le faire en calculant la position des centroïdes de chacun des groupes, puis en les commentant et en les critiquant au vu de vos objectifs.

Donnez une courte liste de pays à cibler, en présentant leurs caractéristiques. Un découpage plus précis qu'en 5 groupes peut si besoin être effectué pour cibler un nombre raisonnable de pays. 

Visualisez vos  partitions dans le premier plan factoriel obtenu par ACP.

Dans votre partition, vous avez obtenu des groupes distincts. Vérifiez donc qu'ils diffèrent réellement. Pour cela, réalisez les tests statistiques suivants :

un test d'adéquation : parmi les 4 variables, ou parmi d'autres variables que vous trouverez pertinentes, trouvez une variable dont la loi est normale ;
un test de comparaison de deux populations (dans le cas gaussien) : choisissez 2 clusters parmi ceux que vous aurez déterminé. Sur ces 2 clusters, testez la variable gaussienne grâce à un test de comparaison.

In [238]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib as plt

In [239]:
#Versions utilisées
print("Jupyter Notebook : " + str(pd.__version__))
print("Pandas : " + str(pd.__version__))
print("Numpy : " + str(np.__version__))
print("Seaborn : " + str(sns.__version__))

Jupyter Notebook : 1.2.4
Pandas : 1.2.4
Numpy : 1.20.1
Seaborn : 0.11.1


# 1. Import et traitement de la population par pays
la variation de la population entre 2013 et 2019, en %, qui sera positive en cas de croissance ou négative en cas de baisse démographique ;

In [240]:
df_population = pd.read_csv('datas/fao-populations_2019- 2013.csv' , header=0, sep=",", decimal=".")
df_population.head(2)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Bilans Alimentaires (2010-),2,Afghanistan,511,Population totale,2501,Population,2013,2013,1000 personnes,32270,*,Chiffre non officiel
1,FBS,Bilans Alimentaires (2010-),2,Afghanistan,511,Population totale,2501,Population,2019,2019,1000 personnes,38042,*,Chiffre non officiel


In [241]:
df_population = df_population[['Code zone', 'Zone', 'Produit', 'Année', 'Valeur']]
df_population['Valeur'] = df_population['Valeur']*1000
df_population.head()

,Code zone,Zone,Produit,Année,Valeur
0,2,Afghanistan,Population,2013,32270000
1,2,Afghanistan,Population,2019,38042000
2,202,Afrique du Sud,Population,2013,53687000
3,202,Afrique du Sud,Population,2019,58558000
4,3,Albanie,Population,2013,2904000


In [242]:
df_population = df_population.pivot_table(index=['Code zone', 'Zone', 'Produit'], columns='Année', values = 'Valeur', aggfunc = sum).reset_index()


In [243]:
# différence de population entre 2013 et 2019, exprimée en pourcentage
df_population['Evolution population (%)'] = round(((df_population[2019]-df_population[2013])/df_population[2013])*100,3)


In [244]:
df_population.head(20)

Année,Code zone,Zone,Produit,2013,2019,Evolution population (%)
0,1,Arménie,Population,2898000.0,2958000.0,2.070
1,2,Afghanistan,Population,32270000.0,38042000.0,17.887
2,3,Albanie,Population,2904000.0,2881000.0,-0.792
3,4,Algérie,Population,38140000.0,43053000.0,12.881
4,7,Angola,Population,26016000.0,31825000.0,22.329
5,8,Antigua-et-Barbuda,Population,92000.0,97000.0,5.435
6,9,Argentine,Population,42196000.0,44781000.0,6.126
7,10,Australie,Population,23255000.0,25203000.0,8.377
8,11,Autriche,Population,8556000.0,8955000.0,4.663
9,12,Bahamas,Population,367000.0,389000.0,5.995


In [245]:
population2013 = df_population[2013].sum()
population2019 = df_population[2019].sum()
evolution_globale = round(((population2019 - population2013) / population2013)*100,3)
print('Pour indication, la population totale en 2013 était de', population2013, 'et de', population2019, 'en 2019.\nL\'évolution 2013/2019 est de', evolution_globale, '%.')



Pour indication, la population totale en 2013 était de 8564484000.0 et de 9108256000.0 en 2019.
L'évolution 2013/2019 est de 6.349 %.


# 2. Données sur les bilans alimentaires mondiaux (2019)


In [246]:
df_dispo_alimentaire = pd.read_csv('datas/FAOSTAT_data.csv', header=0, sep=",", decimal=".")
df_dispo_alimentaire.head(8)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Bilans Alimentaires (2010-),2,Afghanistan,664,Disponibilité alimentaire (Kcal/personne/jour),2901,Total General,2019,2019,Kcal/personne/jour,2273.00,Fc,Donnée calculée
1,FBS,Bilans Alimentaires (2010-),2,Afghanistan,674,Disponibilité de protéines en quantité (g/pers...,2901,Total General,2019,2019,g/personne/jour,57.31,Fc,Donnée calculée
2,FBS,Bilans Alimentaires (2010-),2,Afghanistan,684,Disponibilité de matière grasse en quantité (g...,2901,Total General,2019,2019,g/personne/jour,39.78,Fc,Donnée calculée
3,FBS,Bilans Alimentaires (2010-),2,Afghanistan,664,Disponibilité alimentaire (Kcal/personne/jour),2941,Produits Animaux,2019,2019,Kcal/personne/jour,187.00,Fc,Donnée calculée
4,FBS,Bilans Alimentaires (2010-),2,Afghanistan,674,Disponibilité de protéines en quantité (g/pers...,2941,Produits Animaux,2019,2019,g/personne/jour,10.34,Fc,Donnée calculée
5,FBS,Bilans Alimentaires (2010-),2,Afghanistan,684,Disponibilité de matière grasse en quantité (g...,2941,Produits Animaux,2019,2019,g/personne/jour,12.67,Fc,Donnée calculée
6,FBS,Bilans Alimentaires (2010-),202,Afrique du Sud,664,Disponibilité alimentaire (Kcal/personne/jour),2901,Total General,2019,2019,Kcal/personne/jour,2898.00,Fc,Donnée calculée
7,FBS,Bilans Alimentaires (2010-),202,Afrique du Sud,674,Disponibilité de protéines en quantité (g/pers...,2901,Total General,2019,2019,g/personne/jour,79.77,Fc,Donnée calculée


In [247]:
df_dispo_alimentaire = df_dispo_alimentaire.pivot_table(index=['Code zone', 'Zone', 'Année'],
                                columns=['Élément', 'Produit'],
                                values = 'Valeur',
                                aggfunc=sum).reset_index()
df_dispo_alimentaire.head()

Élément Code zone         Zone Année  \
Produit                                
0               1      Arménie  2019   
1               2  Afghanistan  2019   
2               3      Albanie  2019   
3               4      Algérie  2019   
4               7       Angola  2019   

Élément Disponibilité alimentaire (Kcal/personne/jour)                \
Produit                               Produits Animaux Total General   
0                                                975.0        3022.0   
1                                                187.0        2273.0   
2                                               1021.0        3338.0   
3                                                379.0        3493.0   
4                                                211.0        2445.0   

Élément Disponibilité de matière grasse en quantité (g/personne/jour)  \
Produit                                              Produits Animaux   
0                                                    74.06              
1                                                    12.67              
2                                                    70.01              
3                                                    22.14              
4                                                    15.27              

Élément                \
Produit Total General   
0              115.83   
1               39.78   
2              115.99   
3               98.14   
4               59.42   

Élément Disponibilité de protéines en quantité (g/personne/jour)                
Produit                                         Produits Animaux Total General  
0                                                    52.86               94.29  
1                                                    10.34               57.31  
2                                                    62.47              116.02  
3                                                    24.09               90.84  
4                                                    15.54               52.09

In [248]:
# proportion de protéines d'origine animale par rapport à la quantité totale de protéines dans la disponibilité alimentaire du pays
df_dispo_alimentaire["Prot. Animales/Prot. Totales dispo (%)"] = round((df_dispo_alimentaire[('Disponibilité de protéines en quantité (g/personne/jour)', 'Produits Animaux')]
                                                                         /df_dispo_alimentaire[('Disponibilité de protéines en quantité (g/personne/jour)', 'Total General')])*100,3)
df_dispo_alimentaire.head()

Élément Code zone         Zone Année  \
Produit                                
0               1      Arménie  2019   
1               2  Afghanistan  2019   
2               3      Albanie  2019   
3               4      Algérie  2019   
4               7       Angola  2019   

Élément Disponibilité alimentaire (Kcal/personne/jour)                \
Produit                               Produits Animaux Total General   
0                                                975.0        3022.0   
1                                                187.0        2273.0   
2                                               1021.0        3338.0   
3                                                379.0        3493.0   
4                                                211.0        2445.0   

Élément Disponibilité de matière grasse en quantité (g/personne/jour)  \
Produit                                              Produits Animaux   
0                                                    74.06              
1                                                    12.67              
2                                                    70.01              
3                                                    22.14              
4                                                    15.27              

Élément                \
Produit Total General   
0              115.83   
1               39.78   
2              115.99   
3               98.14   
4               59.42   

Élément Disponibilité de protéines en quantité (g/personne/jour)  \
Produit                                         Produits Animaux   
0                                                    52.86         
1                                                    10.34         
2                                                    62.47         
3                                                    24.09         
4                                                    15.54         

Élément               Prot. Animales/Prot. Totales dispo (%)  
Produit Total General                                         
0               94.29                                 56.061  
1               57.31                                 18.042  
2              116.02                                 53.844  
3               90.84                                 26.519  
4               52.09                                 29.833

# 3.le taux de dépendance aux importations, 
défini en économie comme le rapport entre les importations et la disponibilité intérieure du pays ;

In [249]:
df_dépendance_importations = pd.read_csv('datas/la viande de volailles.csv', header=0, sep=",", decimal=".")
df_dépendance_importations.head(3)

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5511,Production,2734,Viande de Volailles,2019,2019,Milliers de tonnes,28,Im,Données de la FAO basées sur une méthodologie ...
1,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2019,2019,Milliers de tonnes,44,Im,Données de la FAO basées sur une méthodologie ...
2,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5301,Disponibilité intérieure,2734,Viande de Volailles,2019,2019,Milliers de tonnes,67,S,Données standardisées


In [250]:

df_dépendance_importations['Valeur'] = df_dépendance_importations['Valeur']*1000000000
df_dépendance_importations.head(3)


,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5511,Production,2734,Viande de Volailles,2019,2019,Milliers de tonnes,28000000000,Im,Données de la FAO basées sur une méthodologie ...
1,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5611,Importations - Quantité,2734,Viande de Volailles,2019,2019,Milliers de tonnes,44000000000,Im,Données de la FAO basées sur une méthodologie ...
2,FBS,Bilans Alimentaires (2010-),2,Afghanistan,5301,Disponibilité intérieure,2734,Viande de Volailles,2019,2019,Milliers de tonnes,67000000000,S,Données standardisées


In [251]:
df_dépendance_importations = df_dépendance_importations.pivot_table(index=['Code zone', 'Zone', 'Année'],
                                columns=['Élément'],
                                values = 'Valeur',
                                aggfunc=sum).reset_index()
df_dépendance_importations

Élément,Code zone,Zone,Année,Disponibilité intérieure,Importations - Quantité,Production
0,1,Arménie,2019,5.500000e+10,4.300000e+10,1.200000e+10
1,2,Afghanistan,2019,6.700000e+10,4.400000e+10,2.800000e+10
2,3,Albanie,2019,3.400000e+10,2.200000e+10,1.200000e+10
3,4,Algérie,2019,2.870000e+11,0.000000e+00,2.910000e+11
4,7,Angola,2019,3.270000e+11,2.780000e+11,4.900000e+10
...,...,...,...,...,...,...
174,256,Luxembourg,2019,1.200000e+10,1.200000e+10,0.000000e+00
175,272,Serbie,2019,1.120000e+11,1.700000e+10,1.040000e+11
176,273,Monténégro,2019,1.000000e+10,7.000000e+09,4.000000e+09
177,276,Soudan,2019,5.600000e+10,0.000000e+00,7.500000e+10


In [252]:
df_dépendance_importations['taux_dépendance_importations']=df_dépendance_importations['Importations - Quantité']    / df_dépendance_importations['Disponibilité intérieure']*100
df_dépendance_importations.head(2)

Élément,Code zone,Zone,Année,Disponibilité intérieure,Importations - Quantité,Production,taux_dépendance_importations
0,1,Arménie,2019,5.500000e+10,4.300000e+10,1.200000e+10,78.181818
1,2,Afghanistan,2019,6.700000e+10,4.400000e+10,2.800000e+10,65.671642


In [253]:
df_dépendance_importations= df_dépendance_importations[['Code zone','taux_dépendance_importations']]
df_dépendance_importations.head(2)

Élément,Code zone,taux_dépendance_importations
0,1,78.181818
1,2,65.671642


# les valeurs de PIB par habitant,
en $ paritaire, c’est à dire normalisé en fonction du coût de la vie.


In [262]:
# Dataframes des données complémentaires
# Indicateurs Macro (PIB et croissance)
df_indicateurs_macro = pd.read_csv('datas/indicateurs_macro.csv', header=0, sep=',', decimal='.')
df_indicateurs_macro = df_indicateurs_macro[['Code zone (FAO)','Élément','Valeur']]
df_indicateurs_macro = df_indicateurs_macro.pivot_table(index=['Code zone (FAO)'], columns='Élément', values='Valeur', aggfunc=sum).reset_index()
df_indicateurs_macro = df_indicateurs_macro.rename(columns={"Croissance annuelle US$":"Croissance annuelle PIB US$ (%)",
                                    "Valeur US $":"PIB Total US$ (Millions)",'Code zone (FAO)':'Code zone'})
                              
                        


df_indicateurs_macro.head()

Élément,Code zone,Croissance annuelle PIB US$ (%),PIB Total US$ (Millions)
0,1,9.751697,13672.801339
1,2,-0.613895,17876.546427
2,3,0.865170,15278.072762
3,4,-2.426895,171157.863119
4,6,-1.997315,3154.137481


# 3. Construction et sauvegarde du dataframe principal


In [263]:
df_dispo_alimentaire_etendu = pd.merge(df_population, df_dispo_alimentaire, on="Code zone")

df_dispo_alimentaire_etendu.head(2)

/Users/rowaidakanaan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/rowaidakanaan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,Code zone,Zone,Produit,2013,2019,Evolution population (%),"(Zone, )","(Année, )","(Disponibilité alimentaire (Kcal/personne/jour), Produits Animaux)","(Disponibilité alimentaire (Kcal/personne/jour), Total General)","(Disponibilité de matière grasse en quantité (g/personne/jour), Produits Animaux)","(Disponibilité de matière grasse en quantité (g/personne/jour), Total General)","(Disponibilité de protéines en quantité (g/personne/jour), Produits Animaux)","(Disponibilité de protéines en quantité (g/personne/jour), Total General)","(Prot. Animales/Prot. Totales dispo (%), )"
0,1,Arménie,Population,2898000.0,2958000.0,2.070,Arménie,2019,975.0,3022.0,74.06,115.83,52.86,94.29,56.061
1,2,Afghanistan,Population,32270000.0,38042000.0,17.887,Afghanistan,2019,187.0,2273.0,12.67,39.78,10.34,57.31,18.042


In [264]:
df_dataset_fao = df_dispo_alimentaire_etendu[["Code zone","Zone", ('Année', ''), 2019, "Evolution population (%)",
                                             ('Disponibilité alimentaire (Kcal/personne/jour)', 'Total General'),
                                             ('Disponibilité de protéines en quantité (g/personne/jour)', 'Total General'),
                                            ('Prot. Animales/Prot. Totales dispo (%)', '')
]]
df_dataset_fao.rename(columns={('Année', ''):'Année', 2019:'Population', 
                              ('Disponibilité alimentaire (Kcal/personne/jour)', 'Total General'):'Dispo. Alim. (Kcal/personne)',
                              ('Disponibilité de protéines en quantité (g/personne/jour)', 'Total General'):'Dispo. Prot. (g/personne)'},
                     inplace=True)
df_dataset_fao.head()

/Users/rowaidakanaan/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
/Users/rowaidakanaan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Code zone,Zone,Année,Population,Evolution population (%),Dispo. Alim. (Kcal/personne),Dispo. Prot. (g/personne),"(Prot. Animales/Prot. Totales dispo (%), )"
0,1,Arménie,2019,2958000.0,2.070,3022.0,94.29,56.061
1,2,Afghanistan,2019,38042000.0,17.887,2273.0,57.31,18.042
2,3,Albanie,2019,2881000.0,-0.792,3338.0,116.02,53.844
3,4,Algérie,2019,43053000.0,12.881,3493.0,90.84,26.519
4,7,Angola,2019,31825000.0,22.329,2445.0,52.09,29.833


In [265]:
df_dataset_fao = pd.merge(df_dataset_fao, df_dépendance_importations, on="Code zone")
df_dataset_fao = pd.merge(df_dataset_fao, df_indicateurs_macro, on="Code zone")

df_dataset_fao.head(2)

,Code zone,Zone,Année,Population,Evolution population (%),Dispo. Alim. (Kcal/personne),Dispo. Prot. (g/personne),"(Prot. Animales/Prot. Totales dispo (%), )",taux_dépendance_importations,Croissance annuelle PIB US$ (%),PIB Total US$ (Millions)
0,1,Arménie,2019,2958000.0,2.070,3022.0,94.29,56.061,78.181818,9.751697,13672.801339
1,2,Afghanistan,2019,38042000.0,17.887,2273.0,57.31,18.042,65.671642,-0.613895,17876.546427


In [266]:
df_dataset_fao["Dispo. Alim. (Kcal/personne)"] *= 365
df_dataset_fao["Dispo. Prot. (g/personne)"] *= 365